In [11]:
import pandas as pd 
import numpy as np

#reading the given data 
data = pd.read_csv("train.csv") 

#Creating a training data and validation data
ran = np.random.rand(len(data)) < 0.8
train = data[ran]
val = data[~ran]

#Considering only categorical data for part1 of the question
train = train.loc[:,['Work_accident', 'promotion_last_5years', 'sales', 'salary','left'] ]
val = val.loc[:,['Work_accident', 'promotion_last_5years', 'sales', 'salary','left'] ]

#Variable for numberOfNodes(just to have a rough idea)
numberOfNodes = 0

#Class definition for Nodes in Decision Tree
class Node(object):
    #def __init__(self, label,name,parentAttrValue,nodeNumber,level):
    def __init__(self, examples, attributes, decisionAttribute, nodeNumber, parentDecisionAttribute, parentDecisionAttributeValue, decision, level):
        self.examples = examples
        self.attributes = attributes
        self.decisionAttribute = decisionAttribute 
        self.nodeNumber = nodeNumber
        self.children = []
        self.parentDecisionAttribute = parentDecisionAttribute
        self.parentDecisionAttributeValue = parentDecisionAttributeValue
        self.decision = decision
        self.level = level

    def add_child(self, obj):
        self.children.append(obj)
        
    def increment_nodeNumber(self):
        #nodeNumber = nodeNumber+1
        self.nodeNumber = self.nodeNumber + 1
        
    def __str__(self):
        #return self.label +"  "+ self.name+"  "+self.parentAttrValue+"Node number is "+ str(self.nodeNumber)+" of level "+str(self.level)
        return "Decision attribute for this node is "+self.decisionAttribute+" decision "+ self.decision+" Node number is "+ str(self.nodeNumber)+" of level "+str(self.level)+" number of examples of this node "+str(len(self.examples))
    

#Defining the targetAttribute(Label) column, list of attributes is extracted from data, examples refer to samples(rows) in data 
targetAttribute = 'left'
Attributes = list(train.columns)
Attributes.remove(targetAttribute)
AllAttrExpTar = Attributes[:]
examples = list(train.index.values) 

#Function for calculating the total entropy associated with all samples at a node
def getEntropyOfExamples(examplesForCurAttr):
    NumberOfPosEx = 0
    NumberOfNegEx = 0
    for j in examplesForCurAttr:
        if  str(train.at[int(j),targetAttribute]) == str(1):
            NumberOfPosEx = NumberOfPosEx+1
        elif str(train.at[int(j),targetAttribute]) == str(0):
             NumberOfNegEx = NumberOfNegEx+1
    PropOfPos =  NumberOfPosEx/len(examplesForCurAttr)
    PropOfNeg = NumberOfNegEx/len(examplesForCurAttr)
    return ((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))


#Function for calculating the entropy of a attribute for given examples(Used while determining the Best Attribute for a node)
def getEntropyofAttribute(currentAttribute,examplesForCurAttr,targetAttribute):
    NumberOfexamplesForCurAttr = len(examplesForCurAttr)
    sum = 0.0
    fra1 = train.loc[examplesForCurAttr,[currentAttribute] ]
    currentAttributeValues = fra1[currentAttribute].unique()
    for i in currentAttributeValues:
        NumberOfExaForThisValue = 0
        for j in examplesForCurAttr:
            if str(train.at[int(j),currentAttribute]) == str(i):
                NumberOfExaForThisValue = NumberOfExaForThisValue+1
        if NumberOfExaForThisValue == 0:
            sum = sum+0
        else:
            NumberOfPosForThisValue = 0
            NumberOfNegForThisValue = 0
            for j in examplesForCurAttr:
                if str(train.at[int(j),targetAttribute]) == str(1) and str(train.at[int(j),currentAttribute]) == str(i):
                    NumberOfPosForThisValue = NumberOfPosForThisValue+1
                elif str(train.at[int(j),targetAttribute]) == str(0) and str(train.at[int(j),currentAttribute]) == str(i):
                    NumberOfNegForThisValue = NumberOfNegForThisValue+1
            PropOfPos = NumberOfPosForThisValue/NumberOfExaForThisValue
            PropOfNeg = NumberOfNegForThisValue/NumberOfExaForThisValue
            PropOfThisValue = NumberOfExaForThisValue/NumberOfexamplesForCurAttr
            if PropOfPos == 0:
                sum = sum + ((PropOfThisValue)*(-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))
            elif PropOfNeg == 0:
                sum = sum + ((PropOfThisValue)*(-1*PropOfPos*(np.log(PropOfPos)/np.log(2))))
            else:
                sum = sum + ((PropOfThisValue)*((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))))              
    return sum


#Function to calclulate the best attribute at a node
def BestOfAttributes(Attributes,examplesForCurAttr,targetAttribute,method):
    if len(Attributes) == 1:
        return Attributes[0]
    elif method == 'Entropy':
        EntropyForAttributes = {}
        for i in Attributes:
            EntropyForAttributes[i] = getEntropyofAttribute(i,examplesForCurAttr,targetAttribute)
        #findining the minimum among entropy of attributes
        BestOfAttributes = min(EntropyForAttributes, key=EntropyForAttributes.get)
        return BestOfAttributes        
    elif method == 'Gini':
        pass
    elif method == 'misClasifi':
        pass
    return BestOfAttributes

#Function which constructs the decision tree 
def DecisionTreeBuilder(examplesForCurNode,targetAttribute,AttributesForCurrentNode,parentDecisionAttribute,parentAttrValue,level):  
    
    global numberOfNodes
    
    # if all samples are positive(i.e employee left the company,left =1 )
    allExPos = True
    for i in examplesForCurNode:
        if str(train.at[int(i),targetAttribute]) == str(1):
            pass
        else:
            allExPos = False
            break
    if allExPos:
        r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Positive',level)
        numberOfNodes  = numberOfNodes + 1
        return r
    
    # if all samples are negative(i.e employee didnot leave the company,left =0 )
    allExNeg = True
    for j in examplesForCurNode:
        if str(train.at[j,targetAttribute]) == str(0):
            pass
        else:
            allExNeg = False
            break
    if allExNeg:
        r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Negative',level)
        numberOfNodes  = numberOfNodes + 1
        return r
    
    #if attributes are exhausted
    if len(AttributesForCurrentNode) == 0 or getEntropyOfExamples(examplesForCurNode) < 0.01:#or len(examplesForCurNode)<50 :
        fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if (posCount/(posCount+negCount))>0.24:
            r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Positive',level)
            numberOfNodes  = numberOfNodes + 1
        else:
            r = Node(examplesForCurNode,AttributesForCurrentNode,'leafNode',numberOfNodes,parentDecisionAttribute,parentAttrValue,'Negative',level)
            numberOfNodes  = numberOfNodes + 1
        return r
    
    #If none of the above were conditions were true then find the best attribute and create children accordingly
    BestAttribute = BestOfAttributes(AttributesForCurrentNode,examplesForCurNode,targetAttribute,'Entropy')    
    r = Node(examplesForCurNode,AttributesForCurrentNode,BestAttribute,numberOfNodes,parentDecisionAttribute,parentAttrValue,'NotLeaf',level)
    numberOfNodes  = numberOfNodes + 1
    BestAttributeValues = list(data[BestAttribute].unique())
    for iter in BestAttributeValues:
        AttributesForChildren = []
        AttributesForChildren = AttributesForCurrentNode[:]
        AttributesForChildren.remove(BestAttribute)
        examplesForThisChildOfCurrentNode = []
        for iter1 in examplesForCurNode:
            if  str(train.at[int(iter1),str(BestAttribute)]) == str(iter):
                examplesForThisChildOfCurrentNode.append(int(iter1))
        if len(examplesForThisChildOfCurrentNode)==0:
            numberOfNodes  = numberOfNodes + 1
            fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
            targetAttributesList = fra1[targetAttribute].values
            unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
            dictOfTaAttr = dict(zip(unique1, counts1))
            posCount = dictOfTaAttr[1]
            negCount = dictOfTaAttr[0]
            if (posCount/(posCount+negCount))>0.24:
                r.add_child(Node(examplesForThisChildOfCurrentNode,AttributesForChildren,'leafNode',numberOfNodes,BestAttribute,str(iter),'Positive',level+1))
            else:
                r.add_child(Node(examplesForThisChildOfCurrentNode,AttributesForChildren,'leafNode',numberOfNodes,BestAttribute,str(iter),'Negative',level+1))                            
            numberOfNodes = numberOfNodes - 1
        else:        
            r.add_child(DecisionTreeBuilder(examplesForThisChildOfCurrentNode,targetAttribute,AttributesForChildren,BestAttribute,str(iter),level+1))            
    return r

numberOfNodes = 0
examplesForRootNode = examples[:]
AttributesForRootNode = Attributes[:]

#Building the tree from training data 
RootNode = DecisionTreeBuilder(examplesForRootNode,targetAttribute,AttributesForRootNode,'root','root',0)


#Creating a copy of training and validation data to print the predicted value and also calculate precision, recall, accuracy etc
valCopy = val.copy()
valCopy['predict']= [0]*len(valCopy)
trainCopy = train.copy()
trainCopy['predict'] = [0]*len(trainCopy)
trainExamples = list(train.index.values)
valExamples = list(val.index.values)

#function definition to predict on given TestSet
def RecursiveFunctionToPredict(i,r,TestSet):
    if r.decisionAttribute == 'leafNode':
        if r.decision == 'Positive':
            return '1'
        else:
            return '0'
    else:
        AttrValue = str(TestSet.at[int(i),str(r.decisionAttribute)])
        for j in r.children:
            if str(j.parentDecisionAttributeValue) == str(AttrValue):
                break
        return RecursiveFunctionToPredict(i,j,TestSet)

def predict(RootNode,NameOfTestFile):
    GivenTestData = pd.read_csv(str(NameOfTestFile))
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    return GivenTestData
Prediction = predict(RootNode,'sample_test.csv')
Prediction.head()   

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,predict
0,0.69,0.69,3,236,4,0,0,product_mng,medium,0
1,0.36,0.54,2,153,3,1,0,accounting,medium,0


In [ ]:
TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in list(GivenTestData.index.values):
        if str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(1):
            TP = TP + 1
        elif str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(0):
            FN = FN + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(1):
            FP = FP + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(0):
            TN = TN + 1
    print(TN)
    print(FP)
    print(FN)
    print(TP)
    Precision = (TP/(TP+FP))
    Recall = (TP/(TP+FN))
    print(Precision)
    print(Recall)
    F1_Score = 2*((1/Recall)+(1/Precision))
    print(F1_Score)
    accuracy = (TN+TP)/(TN+TP+FP+FN)
    print(accuracy)